In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import dill

import TwoPUtils as tpu
import STX3KO_analyses as stx
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

/home/mplitt/anaconda3/envs/STX3_test/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = 'mCherry9'

file_list = stx.ymaze_sess_deets.CTRL_sessions[mouse]
for f in file_list:
    print(f)


{'date': '15_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 0}
{'date': '16_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 1}
{'date': '17_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 2}
{'date': '19_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 3}
{'date': '20_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 4}
{'date': '21_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 5}
({'date': '22_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 6}, {'date': '22_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4, 'novel_arm': 1, 'ravel_ind': 7})
{'date': '22_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 4, 'novel_arm': 1, 'ravel_ind': 8}
({'date': '23_10_2021', 'scene': 'YM

In [3]:
def check_and_make_h5s(file_list, runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        source_folder  = os.path.join('/mnt','BigDisk','2P_scratch')
        source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
        info = tpu.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ =  tpu.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=False)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [4]:
def run_session(fdict):
    _h5list,_frs = check_and_make_h5s([fdict,],runh5=True)
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')

    ops = tpu.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 


for f in file_list[-1:]: #file_list:
    
    if isinstance(f,tuple):
        for _f in f:
            run_session(_f)
    else:
        run_session(f)
    

{'date': '23_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 9}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 10575
['/mnt/BigDisk/2P_scratch/mCherry9/23_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_002']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/mCherry9/23_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_002/YMaze_RewardReversal_001_002.h5']
time 658.86 sec. Wrote 10575 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 10575 frames
Reference frame, 24.22 sec.
Registered 2000/10575 in 108.59s
Registered 4000/10575 in 208.29s
Registered 6000/10575 in 301.33s
Registered 8000/10575 in 394.01s
Registered 10000/10575 in 484.9

In [4]:
def update_sess_dict(mouse, f):
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])

            
    source_folder = "/mnt/BigDisk/2P_scratch"
    source_stem = os.path.join(source_folder, mouse, f['date'], f['scene'], scan_str)

    f['mouse']=mouse
    f.update({'scan_file': source_stem + '.sbx',
              'scanheader_file': source_stem + '.mat',
              'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
              'scan_number': f['scan'],
              'prompt_for_keys': False,
              'VR_only': False,
              'scanner': "NLW",
             })
    return f

def run_and_save(fdict,comb_frames=0):
#     with open(os.path.join('/home/mplitt/YMazeSessPkls',mouse,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
#         sess = dill.load(file)
#     sess.novel_arm = fdict['novel']    
            
    sess = stx.session.YMazeSession(**fdict)
    sess.load_scan_info()
    sess.align_VR_to_2P()
    sess.load_suite2p_data(which_ts=('F', 'Fneu','spks'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
    sess.get_trial_info()
    sess.neuropil_corrected_dff()
    
    
    sess.place_cells_calc(nperms=1000)
    
    sess.add_pos_binned_trial_matrix('F_dff')
    sess.add_timeseries(F_dff_norm = sess.timeseries['F_dff']/np.nanpercentile(sess.timeseries['F_dff'],99,axis=1,keepdims=True))
    sess.rm_pos_binned_trial_matrix('F_dff_norm')
    sess.add_pos_binned_trial_matrix('F_dff_norm')
    
    sess.add_pos_binned_trial_matrix('spks') #,speed=sess.vr_data['dz']._values, speed_thr = .05)  
    sess.add_timeseries(spks_norm = sess.timeseries['spks']/np.nanpercentile(sess.timeseries['spks'],99,axis=1,keepdims=True))
    sess.add_pos_binned_trial_matrix('spks_norm') 

    sess.add_timeseries(licks=sess.vr_data['lick']._values)
    sess.add_pos_binned_trial_matrix('licks')
    sess.add_timeseries(speed=sess.vr_data['dz']._values)
    sess.add_pos_binned_trial_matrix('speed')
    
    tpu.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')
    return comb_frames+sess.scan_info['max_idx']

In [5]:
for f in file_list: #file_list:
    print(f)
    if isinstance(f,tuple):
        for _f in f:
            f_ = update_sess_dict(mouse,_f)
            _ = run_and_save(f_)
    else:
        f_ = update_sess_dict(mouse,f) 
        _ = run_and_save(f_)

{'date': '15_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 0}
Fixing teleports
(127421,) (26342,)
1698.6739294710328 1698.56322418136
/mnt/BigDisk/2P_scratch/mCherry9/15_10_2021/YMaze_LNovel/YMaze_LNovel_001_003/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


233 3372
4310 7466
8402 11700
12635 15815
16750 19868
20803 26341
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '16_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 1}
Fixing teleports
(119680,) (24211,)
1561.288161209068 1561.1486146095717
/mnt/BigDisk/2P_scratch/mCherry9/16_10_2021/YMaze_LNovel/YMaze_LNovel_001_003/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


220 3567
4505 7493
8428 11326
12260 14838
15774 18408
19356 24210
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '17_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 2}
Fixing teleports
(135370,) (28017,)
1806.692065491184 1806.5732997481107
/mnt/BigDisk/2P_scratch/mCherry9/17_10_2021/YMaze_LNovel/YMaze_LNovel_001_002/suite2p


...

/mnt/BigDisk/2P_scratch/mCherry9/22_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_002_004/suite2p
196 5030
5964 10612
11550 16343
17284 22046
22982 27686
28625 38069
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
({'date': '23_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 9}, {'date': '23_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4, 'novel_arm': 1, 'ravel_ind': 10})
Fixing teleports
(50631,) (10575,)
681.9670025188917 681.8498740554156
/mnt/BigDisk/2P_scratch/mCherry9/23_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_002/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


175 4953
5892 10574
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
Fixing teleports
(127988,) (26724,)
1723.2823677581864 1723.1959697732998
/mnt/BigDisk/2P_scratch/mCherry9/23_10_2021/YMaze_LNovel/YMaze_LNovel_001_004/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


191 4995
5933 10865
11802 16460
17401 26723
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


In [3]:
def single_mouse_aligner(mouse, sessions):
    sess_dir = os.path.join('/home/mplitt/YMazeSessPkls',mouse)
    sess_list = []
    for sess_deets in sessions:     
        if isinstance(sess_deets,(list,tuple)):
            for _sess_deets in sess_deets:
                with open(os.path.join(sess_dir,_sess_deets['date'],"%s_%d.pkl" % (_sess_deets['scene'], _sess_deets['session'])),'rb') as file:
                    sess_list.append(dill.load(file))
        else:
            with open(os.path.join(sess_dir,sess_deets['date'],"%s_%d.pkl" % (sess_deets['scene'], sess_deets['session'])),'rb') as file:
                    sess_list.append(dill.load(file))

    sa = tpu.roi_matching.ROIAligner(sess_list)
    sa.run_pairwise_matches()

    with open(os.path.join(sess_dir,'roi_aligner_results.pkl'), 'wb') as file:
        dill.dump(sa.match_inds,file)

In [4]:
mouse = 'mCherry9'
sessions = stx.ymaze_sess_deets.CTRL_sessions[mouse]
single_mouse_aligner(mouse,sessions)

thresh 0.30266471004486084
thresh 0.30350874161720276
thresh 0.3352069983482361
thresh 0.28385157680511475
thresh 0.32532693243026733
thresh 0.3076258430480957
thresh 0.3265962133407593
thresh 0.3158455023765564
thresh 0.32139499282836914
thresh 0.3233503828048706
thresh 0.37092913484573364
thresh 0.3525808880329132
thresh 0.37435214018821716
thresh 0.38547895646095276
thresh 0.33544684743881226
thresh 0.35971294140815735
thresh 0.32222457027435303
skipping roi
thresh 0.3734116086959839
thresh 0.3357828984260559
thresh 0.3518719801902771
thresh 0.35780311918258667
thresh 0.3460416922569275
thresh 0.32119415497779846
thresh 0.3507094213962555
thresh 0.3435680994987488
thresh 0.29564712738990784
thresh 0.32812215185165405
thresh 0.33537326550483704
thresh 0.34655140137672424
thresh 0.345520628452301
thresh 0.3217570016384125
thresh 0.3647133836746216
thresh 0.29432926869392395
thresh 0.3477312157154083
thresh 0.305654598236084
thresh 0.31410364603996277
thresh 0.31037886238098145
thresh 